## OLS Model

In [1]:
## Laading libraries
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.patches as mpatches
from scipy.optimize import minimize
from pandas_datareader import DataReader
from datetime import datetime
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [2]:
# # 2019 stock data for P&G from Yahoo:
# PG = DataReader('PG',  'yahoo', datetime(2019,1,1), datetime(2019,12,31));

# # # Avergae of opening and closing:
# PG["Average"] = (PG["Open"] + 2*PG["High"] + PG["Close"])/4
# PG["Days"] = range(1,PG.shape[0]+1)

# # Required Dataset: 
# print(PG.shape)
# print(PG.head())

In [3]:
# loading dataset
def getData(strt_date, end_date):
    FF = pd.read_csv("C:\\Users\\sshai\\Desktop\\Northeastern University\\INFO 7374 ML In Finance\\INFO-7374-Machine-Learning-in-Finance\\Dashboard\\Shaishav\\data.csv")
    # Date format change
    FF["Date"] = pd.to_datetime(FF["Date"])
    # Data selection
    df = FF[(FF["Date"] >= strt_date) & (FF["Date"] <= end_date)]
    # Reset Index
    df = df.reset_index()
    df = df.drop(["index"], axis =1)
    # Average Column and no. of days calculations
    df["Average"] = (FF["Open"] + 2*FF["High"] + FF["Close"])/4
    df["Days"] = range(1,df.shape[0]+1)
    df["beta_0"] = 1
    return df

In [4]:
def variable_correlation(df):
    fig, ax = plt.subplots(figsize=(20,20))
    sns.heatmap(df.corr(), center=0, cmap="BrBG", annot=True)

In [5]:
def beta_values(df, colm):
    x_t = np.array(df[colm].shift(periods=-1).dropna())
    y_t = np.array(df[["Open"]].shift(periods=1).dropna())
    b_t = np.matmul(np.matmul(np.linalg.inv(np.matmul(x_t.T,x_t)),x_t.T),y_t)
    b_t = b_t.tolist()
    b_t = [item for sublist in b_t for item in sublist]
    return b_t

In [6]:
def predictions(beta, df, colm):
    x_t = np.array(df[colm].shift(periods=-1).dropna())
    b_t = beta
    y_pred = np.matmul(x_t,b_t)
    return y_pred

In [7]:
def error_cal(ypred,df):
    y_pred = ypred
    y_act = df['Open'][:-1]
    error = list(y_act - y_pred)
    return error

In [8]:
def residual_variance(error):
    SSE = sum(map(lambda i : i * i, error)) 
    return SSE    

In [24]:
def totalError_Square(df):
    y_act = df['Open'][:-1]
    n = len(df["Open"][:-1])
    y_mean = sum(df["Open"])/n
    res = list(y_act - y_mean)
    SST = sum(map(lambda i : i * i, res))
    return SST

In [29]:
def R_squared(error, df):
    SSE = residual_variance(error)
    SSR = totalError_Square(df_main)
    r2 = 1 - float(SSE/SSR)
    return r2 

In [39]:
## User input, can be changed
strt_date = "2009-01-01"
end_date = "2019-12-31"

## final dataset
df_main = getData(strt_date, end_date)
# print(df_main.head())
# print(df_main.shape)
# plt.scatter(df_main["Days"],df_main["Open"])

## EDA
# variable_correlation(df_main)


## Variable selection for OLS:
colm = ['beta_0','RF', 'RMW', 'Days', "Average","Volume","SMB","Mkt-RF","HML","CMA"]
colm = ['beta_0','RF', 'Average','CMA', 'SMB', "HML", "Days", "Volume"]

## Calculating beta_values:
b_t = beta_values(df_main, colm)

# Sorting variables based on their beta values
df_beta = pd.DataFrame(list(zip(colm, b_t)), columns = ["Variables","Beta_Coeff"])
df_beta = df_beta.sort_values(by=['Beta_Coeff'], ascending=False) 
print(df_beta)

# Calculating Error using beta values found
y_pred = predictions(b_t, df_main, colm)
e = error_cal(y_pred, df_main)

# Calculating accuracy estimators:
r2 = R_squared(e, df_main)
r2

  Variables    Beta_Coeff
1        RF  6.823441e+02
0    beta_0  4.389818e+01
2   Average  3.593348e-01
3       CMA  2.389187e-01
4       SMB  5.935954e-02
5       HML  1.562908e-02
6      Days  1.093479e-02
7    Volume -1.694881e-07


0.7946899865186544